## IMPORT LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from math import floor
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

## PARAMETER GRID

In [2]:
slow_range = range(10, 31, 2)
fast_range = range(5, 21, 2)
smooth_range = range(5, 16, 2)

## Create a DataFrame to store combinations of parameters

In [3]:
param_grid = pd.DataFrame([(slow, fast, smooth) for slow in slow_range for fast in fast_range for smooth in smooth_range],
                          columns=['slow', 'fast', 'smooth'])

## FUNCTION TO CALCULATE MACD

In [4]:
def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span=fast, adjust=False).mean()
    exp2 = price.ewm(span=slow, adjust=False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns={'Close': 'macd'})
    signal = pd.DataFrame(macd.ewm(span=smooth, adjust=False).mean()).rename(columns={'macd': 'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns={0: 'hist'})
    return macd, signal, hist

## EXTRACTING STOCK DATA

In [8]:
def get_historical_data(symbol):
    df = yf.Ticker(symbol)
    df = df.history(period="max")
    print(df)
    print(df.index)
    del df["Dividends"]
    del df["Stock Splits"]
    print(df)

    traindf = df[df.index < "2021-01-01"]
    validationdf = df[df.index < "2023-01-01"]
    validationdf = validationdf[validationdf.index >= "2021-01-01"]
    testdf = df[df.index >= "2023-01-01"]
    print(traindf)
    print(validationdf)
    print(testdf)

    return traindf, validationdf, testdf

traindf, validationdf, testdf = get_historical_data(stock_symbol)
print(traindf)
print(validationdf)
print(testdf)

## Prepare your data and create feature-target pairs

In [9]:
def prepare_data(symbol):
    data = get_historical_data(symbol)
    X = []
    y = []

    for i in range(len(data)):
        close = data['Close'][:i+1]  # Adjust based on your need for historical data
        for slow, fast, smooth in zip(param_grid['slow'], param_grid['fast'], param_grid['smooth']):
            _, _, hist = get_macd(close, slow, fast, smooth)
            X.append(hist.values.flatten())
            y.append((slow, fast, smooth))

    return np.array(X), np.array(y)

## Split the data into training and testing sets

In [ ]:
symbol = input("Enter the Stock Symbol (^NSEI):  ");
X, y = prepare_data(symbol)
print(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Enter the Stock Symbol (^NSEI):   ^NSEI


## Convert data to PyTorch tensors

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

## CNN model

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=3)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * (X_train.shape[1] - 2), 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 3)  # 3 parameters for slow, fast, smooth

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        return x

model = CNN()

## Define loss function and optimizer

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Create DataLoader for training

In [ ]:
train_dataset = TensorDataset(X_train_tensor.unsqueeze(1), y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

## Train the model

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

## Evaluate the model

In [ ]:
with torch.no_grad():
    outputs = model(X_test_tensor.unsqueeze(1))
    test_loss = criterion(outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item()}')

## Predict optimal parameters

In [ ]:
predicted_params = model(X_test_tensor.unsqueeze(1))

## Visualize the predicted parameters and compare with actual parameters

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_test[:, 0], label='Actual Slow', color='blue')
plt.plot(predicted_params[:, 0].detach().numpy(), label='Predicted Slow', color='red')
plt.plot(y_test[:, 1], label='Actual Fast', color='green')
plt.plot(predicted_params[:, 1].detach().numpy(), label='Predicted Fast', color='orange')
plt.plot(y_test[:, 2], label='Actual Smooth', color='purple')
plt.plot(predicted_params[:, 2].detach().numpy(), label='Predicted Smooth', color='brown')
plt.legend()
plt.title('Optimized MACD Parameters Prediction')
plt.xlabel('Sample')
plt.ylabel('Parameter Value')
plt.show()